# ⚠️ Уведомление о миграции

Этот notebook был автоматически обновлен для работы с новой модульной архитектурой SAMe:

- 🧹 **same_clear** - Обработка и очистка текста
- 🔍 **same_search** - Поиск и индексация
- 🌐 **same_api** - API и интеграции

📚 См. [MIGRATION_GUIDE.md](../../MIGRATION_GUIDE.md) для подробной информации.

---


# Проект SAMe: Комплексный анализ системы поиска аналогов

## 📊 Анализ основного датасета для оптимизации поиска аналогов

Данный notebook предоставляет комплексную аналитическую платформу для системы поиска аналогов проекта SAMe (Search Analog Model Engine). Анализ сосредоточен на каталоге `main_dataset.xlsx` для оптимизации производительности поиска, сопоставления по сходству и категоризации.

### 🎯 Цели анализа:
1. **Обзор данных и оценка качества** - Структура датасета и метрики качества
2. **Текстовый анализ для оптимизации поиска** - Предобработка названий/описаний товаров
3. **Метрики сходства и сопоставления** - Сравнение алгоритмов и оптимизация
4. **Анализ категоризации и кластеризации** - Группировка и классификация товаров
5. **Оптимизация производительности поиска** - Структура индексов и рекомендации по производительности
6. **Визуализация и выводы** - Интерактивные визуализации и результаты

---

## 🔧 Настройка и импорты

In [1]:
# Основные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Обработка текста
import re
from collections import Counter, defaultdict
import string

# NLP библиотеки
try:
    import spacy
    nlp = spacy.load('ru_core_news_sm')
    SPACY_AVAILABLE = True
except:
    SPACY_AVAILABLE = False
    print("⚠️ Русская модель SpaCy недоступна. Некоторые NLP функции будут ограничены.")

# Библиотеки для сходства и машинного обучения
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Нечеткое сопоставление
from rapidfuzz import fuzz, process
from fuzzywuzzy import fuzz as fuzz_wuzzy

# Визуализация
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

# Импорты проекта SAMe
import sys
sys.path.append('../../src')

try:
    from same.data_manager import data_helper
    from same_clear.text_processing.text_cleaner import TextCleaner, CleaningConfig
    from same_clear.text_processing.lemmatizer import Lemmatizer, LemmatizerConfig
    from same_clear.text_processing.normalizer import TextNormalizer, NormalizerConfig
    SAME_MODULES_AVAILABLE = True
    print("✅ Модули SAMe успешно загружены")
except ImportError as e:
    SAME_MODULES_AVAILABLE = False
    print(f"⚠️ Модули SAMe недоступны: {e}")

# Конфигурация
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("🚀 Настройка завершена успешно!")

✅ Модули SAMe успешно загружены
🚀 Настройка завершена успешно!


## 📂 Загрузка данных и первичное исследование

In [2]:
# Загрузка основного датасета
dataset_path = Path('../../src/data/input/main_dataset.xlsx')

if not dataset_path.exists():
    raise FileNotFoundError(f"Датасет не найден по пути {dataset_path}")

print(f"📂 Загрузка датасета из: {dataset_path}")
df = pd.read_excel(dataset_path)

print(f"✅ Датасет загружен успешно!")
print(f"📊 Размер: {df.shape[0]:,} строк × {df.shape[1]} столбцов")
print(f"💾 Использование памяти: {df.memory_usage(deep=True).sum() / 1024**2:.2f} МБ")

📂 Загрузка датасета из: ../../src/data/input/main_dataset.xlsx
✅ Датасет загружен успешно!
📊 Размер: 130,303 строк × 6 столбцов
💾 Использование памяти: 89.13 МБ


## 1️⃣ Обзор данных и оценка качества

### 1.1 Анализ структуры датасета

In [3]:
# Основная информация о датасете
print("=== СТРУКТУРА ДАТАСЕТА ===")
print(f"Столбцы ({len(df.columns)}): {list(df.columns)}")
print(f"\nТипы данных:")
print(df.dtypes)

# Отображение первых строк
print(f"\n=== ОБРАЗЕЦ ДАННЫХ ===")
display(df.head())

=== СТРУКТУРА ДАТАСЕТА ===
Столбцы (6): ['Код', 'Наименование', 'НаименованиеПолное', 'Группа', 'ВидНоменклатуры', 'ЕдиницаИзмерения']

Типы данных:
Код                   object
Наименование          object
НаименованиеПолное    object
Группа                object
ВидНоменклатуры       object
ЕдиницаИзмерения      object
dtype: object

=== ОБРАЗЕЦ ДАННЫХ ===


,Код,Наименование,НаименованиеПолное,Группа,ВидНоменклатуры,ЕдиницаИзмерения
0,НИ-IS0032430,Cветильник LED панель 50W 6500k IP40 1200мм ДП...,Cветильник LED панель 50W 6500k IP40 1200мм ДП...,АВАНСОВЫЙ ОТЧЕТ,Инвентарь и хозяйственные принадлежности,шт
1,НИ-IS0040225,"Cветильник потолочный,Osairous,Белый, 30Вт LED...","Cветильник потолочный,Osairous,Белый, 30Вт LED...",АВАНСОВЫЙ ОТЧЕТ,Сырье и материалы,шт
2,НИ-IS0014483,Cистема IP-DECT Yealink W80DM контроллер микро...,Cистема IP-DECT Yealink W80DM контроллер микро...,АВАНСОВЫЙ ОТЧЕТ,Инвентарь и хозяйственные принадлежности,шт
3,НИ-IS0047648,Cпрей для ванной комнаты Sanfor (Санфор) 500мл,Cпрей для ванной комнаты Sanfor (Санфор) 500мл,АВАНСОВЫЙ ОТЧЕТ,Сырье и материалы,шт
4,НИ-IS0047649,Cпрей ультрабелый Sanfor (Санфор) 500мл,Cпрей ультрабелый Sanfor (Санфор) 500мл,АВАНСОВЫЙ ОТЧЕТ,Сырье и материалы,шт


In [4]:
# Определение ключевых столбцов для поиска аналогов
key_columns = {
    'product_name': ['Наименование', 'Наименование МТР', 'Name', 'Product_Name'],
    'category': ['Группа', 'Group', 'Category'],
    'subcategory': ['ВидНоменклатуры', 'Product_Type', 'Subcategory'],
    'description': ['Описание', 'Description', 'Desc'],
    'unit': ['ЕдиницаИзмерения', 'Unit', 'Measure']
}

found_columns = {}
for key, possible_names in key_columns.items():
    for name in possible_names:
        if name in df.columns:
            found_columns[key] = name
            break
    if key not in found_columns:
        found_columns[key] = None

print("=== СОПОСТАВЛЕНИЕ КЛЮЧЕВЫХ СТОЛБЦОВ ===")
for key, column in found_columns.items():
    status = "✅" if column else "❌"
    key_ru = {
        'product_name': 'название_товара',
        'category': 'категория', 
        'subcategory': 'подкategория',
        'description': 'описание',
        'unit': 'единица_измерения'
    }
    print(f"{status} {key_ru.get(key, key)}: {column}")

# Сохранение основных столбцов для анализа
MAIN_NAME_COL = found_columns['product_name']
CATEGORY_COL = found_columns['category']
SUBCATEGORY_COL = found_columns['subcategory']

# Проверка доступности данных для анализа
if MAIN_NAME_COL:
    print(f"\n🔍 ПРОВЕРКА ДАННЫХ:")
    print(f"Основной столбец: {MAIN_NAME_COL}")
    if MAIN_NAME_COL in df.columns:
        sample_data = df[MAIN_NAME_COL].dropna().head(3)
        print(f"Примеры данных:")
        for i, item in enumerate(sample_data, 1):
            print(f"  {i}. {item}")
    else:
        print(f"❌ Столбец {MAIN_NAME_COL} не найден!")
else:
    print(f"\n❌ Основной столбец с названиями товаров не определен")

=== СОПОСТАВЛЕНИЕ КЛЮЧЕВЫХ СТОЛБЦОВ ===
✅ название_товара: Наименование
✅ категория: Группа
✅ подкategория: ВидНоменклатуры
❌ описание: None
✅ единица_измерения: ЕдиницаИзмерения

🔍 ПРОВЕРКА ДАННЫХ:
Основной столбец: Наименование
Примеры данных:
  1. Cветильник LED панель 50W 6500k IP40 1200мм ДПО-108 Призма Neox
  2. Cветильник потолочный,Osairous,Белый, 30Вт LED, 6500K Светодиодная потолочная
  3. Cистема IP-DECT Yealink W80DM контроллер микросота DECT


### 1.2 Метрики качества данных

In [5]:
# Анализ пропущенных значений
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

quality_df = pd.DataFrame({
    'Количество_пропусков': missing_data,
    'Процент_пропусков': missing_percent,
    'Тип_данных': df.dtypes
})
quality_df = quality_df[quality_df['Количество_пропусков'] > 0].sort_values('Процент_пропусков', ascending=False)

print("=== АНАЛИЗ ПРОПУЩЕННЫХ ДАННЫХ ===")
if len(quality_df) > 0:
    display(quality_df)
else:
    print("✅ Пропущенные данные не найдены!")

# Анализ дубликатов
if MAIN_NAME_COL:
    # Убедимся, что столбец существует и содержит данные
    if MAIN_NAME_COL in df.columns:
        # Очистим данные от NaN и пустых строк
        clean_names = df[MAIN_NAME_COL].dropna().astype(str)
        clean_names = clean_names[clean_names.str.strip() != '']
        
        total_products = len(clean_names)
        unique_names = clean_names.nunique()
        duplicates = total_products - unique_names
        duplicate_rate = (duplicates / total_products) * 100 if total_products > 0 else 0
        
        print(f"\n=== АНАЛИЗ ДУБЛИКАТОВ ===")
        print(f"Всего товаров (после очистки): {total_products:,}")
        print(f"Уникальных названий: {unique_names:,}")
        print(f"Дубликатов: {duplicates:,} ({duplicate_rate:.2f}%)")
        
        if duplicates > 0 and total_products > 0:
            # Показать наиболее частые дубликаты
            duplicate_names = clean_names.value_counts().head(15)
            duplicate_names = duplicate_names[duplicate_names > 1]
            
            if len(duplicate_names) > 0:
                print(f"\nТоп дублирующихся названий товаров:")
                for name, count in duplicate_names.items():
                    print(f"  • {name}: {count} вхождений")
            else:
                print(f"\n✅ Точных дубликатов не найдено!")
        else:
            print(f"\n✅ Дубликаты не обнаружены!")
    else:
        print(f"\n❌ Столбец {MAIN_NAME_COL} не найден в датасете")
else:
    print(f"\n❌ Основной столбец с названиями товаров не определен")

=== АНАЛИЗ ПРОПУЩЕННЫХ ДАННЫХ ===


,Количество_пропусков,Процент_пропусков,Тип_данных
НаименованиеПолное,108,0.082884,object
ЕдиницаИзмерения,25,0.019186,object



=== АНАЛИЗ ДУБЛИКАТОВ ===
Всего товаров (после очистки): 130,303
Уникальных названий: 128,937
Дубликатов: 1,366 (1.05%)

Топ дублирующихся названий товаров:
  • Выключатель автоматический модульный ВА47-100-3С80-УХЛ3-КЭАЗ 141629: 22 вхождений
  • Емкость пластиковая под питьевую воду 50 л: 20 вхождений
  • Вентилятор вытяжной Blauberg Aero Vintage 125: 19 вхождений
  • Ключ шарнирный КШС 219-245: 8 вхождений
  • Ключ трубный цепной облегченный КЦО-1: 8 вхождений
  • Насос электрический погружной PETROLL Spire 24V (52мм): 7 вхождений
  • Штроп эксплуатационный ШЭ-32: 6 вхождений
  • Ключ шарнирный КШС 73-89 ТУ 342216-75: 6 вхождений
  • Сапоги кожаные с жестким подноском: 6 вхождений
  • Ключ шарнирный КШС-108-127: 6 вхождений
  • БелАК Насос перекачки топлива помповый 24V "Стандарт" комплект: фильтр грубой очистки, 3 хомута, болты крепления БАК.11024: 6 вхождений
  • Агрегат насосный консольный К65-50-160 с электродвигателем 5,5 кВт: 5 вхождений
  • Пистолет для монтажной пены: 5 вхож

### 1.3 Визуализация качества данных

In [6]:
# Создание визуализаций для оценки качества
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Пропущенные данные по столбцам', 'Распределение типов данных', 
                   'Анализ дубликатов', 'Полнота столбцов'),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# График пропущенных данных
if len(quality_df) > 0:
    fig.add_trace(
        go.Bar(x=quality_df.index, y=quality_df['Процент_пропусков'],
               name='Пропуски %', marker_color='red'),
        row=1, col=1
    )

# Распределение типов данных
dtype_counts = df.dtypes.value_counts()
fig.add_trace(
    go.Pie(labels=dtype_counts.index.astype(str), values=dtype_counts.values,
           name="Типы данных"),
    row=1, col=2
)

# График дубликатов
if MAIN_NAME_COL and MAIN_NAME_COL in df.columns:
    # Подготовка данных для анализа дубликатов
    clean_names_for_viz = df[MAIN_NAME_COL].dropna().astype(str)
    clean_names_for_viz = clean_names_for_viz[clean_names_for_viz.str.strip() != '']
    
    if len(clean_names_for_viz) > 0:
        duplicate_counts = clean_names_for_viz.value_counts()
        duplicate_stats = {
            'Уникальные': sum(duplicate_counts == 1),
            'Дубликаты (2-5)': sum((duplicate_counts >= 2) & (duplicate_counts <= 5)),
            'Дубликаты (6-10)': sum((duplicate_counts >= 6) & (duplicate_counts <= 10)),
            'Дубликаты (>10)': sum(duplicate_counts > 10)
        }
        
        fig.add_trace(
            go.Bar(x=list(duplicate_stats.keys()), y=list(duplicate_stats.values()),
                   name='Дубликаты', marker_color='orange'),
            row=2, col=1
        )

# Полнота столбцов
completeness = ((len(df) - missing_data) / len(df)) * 100
fig.add_trace(
    go.Bar(x=df.columns, y=completeness,
           name='Полнота %', marker_color='green'),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=False, title_text="Панель оценки качества данных")
fig.show()

## 2️⃣ Текстовый анализ для оптимизации поиска

### 2.1 Анализ предобработки текста названий товаров

In [7]:
if not MAIN_NAME_COL:
    print("❌ Основной столбец с названиями товаров не найден. Пропускаем текстовый анализ.")
else:
    # Извлечение названий товаров для анализа
    product_names = df[MAIN_NAME_COL].dropna().astype(str)
    
    print(f"=== ОБЗОР ТЕКСТОВОГО АНАЛИЗА ===")
    print(f"Всего названий товаров: {len(product_names):,}")
    print(f"Уникальных названий товаров: {product_names.nunique():,}")
    print(f"Средняя длина названия: {product_names.str.len().mean():.1f} символов")
    print(f"Медианная длина названия: {product_names.str.len().median():.1f} символов")
    
    # Статистика символов и слов
    name_lengths = product_names.str.len()
    word_counts = product_names.str.split().str.len()
    
    print(f"\n=== СТАТИСТИКА ДЛИНЫ ===")
    print(f"Минимальная длина: {name_lengths.min()} символов")
    print(f"Максимальная длина: {name_lengths.max()} символов")
    print(f"Стандартное отклонение: {name_lengths.std():.1f} символов")
    print(f"\nСреднее количество слов в названии: {word_counts.mean():.1f}")
    print(f"Максимальное количество слов в названии: {word_counts.max()}")

=== ОБЗОР ТЕКСТОВОГО АНАЛИЗА ===
Всего названий товаров: 130,303
Уникальных названий товаров: 128,937
Средняя длина названия: 49.0 символов
Медианная длина названия: 44.0 символов

=== СТАТИСТИКА ДЛИНЫ ===
Минимальная длина: 3 символов
Максимальная длина: 150 символов
Стандартное отклонение: 24.1 символов

Среднее количество слов в названии: 6.3
Максимальное количество слов в названии: 31


In [8]:
# Анализ текстовых паттернов
if MAIN_NAME_COL:
    # Паттерны определения языка
    cyrillic_pattern = re.compile(r'[а-яё]', re.IGNORECASE)
    latin_pattern = re.compile(r'[a-z]', re.IGNORECASE)
    numeric_pattern = re.compile(r'\d')
    special_chars_pattern = re.compile(r'[^\w\s]')
    
    cyrillic_count = product_names.str.contains(cyrillic_pattern).sum()
    latin_count = product_names.str.contains(latin_pattern).sum()
    numeric_count = product_names.str.contains(numeric_pattern).sum()
    special_chars_count = product_names.str.contains(special_chars_pattern).sum()
    
    print(f"=== АНАЛИЗ ТЕКСТОВЫХ ПАТТЕРНОВ ===")
    print(f"Названия с кириллицей: {cyrillic_count:,} ({cyrillic_count/len(product_names)*100:.1f}%)")
    print(f"Названия с латиницей: {latin_count:,} ({latin_count/len(product_names)*100:.1f}%)")
    print(f"Названия с цифрами: {numeric_count:,} ({numeric_count/len(product_names)*100:.1f}%)")
    print(f"Названия со спецсимволами: {special_chars_count:,} ({special_chars_count/len(product_names)*100:.1f}%)")
    
    # Общие паттерны
    print(f"\n=== ОБЩИЕ ПАТТЕРНЫ ===")
    
    # Паттерны размеров/измерений
    size_patterns = [
        (r'\d+[xх]\d+', 'Размеры (NxN)'),
        (r'\d+\s*мм', 'Миллиметры'),
        (r'\d+\s*см', 'Сантиметры'),
        (r'\d+\s*м(?!м)', 'Метры'),
        (r'М\d+', 'М-размер (М10, М12, и т.д.)'),
        (r'\d+\s*кг', 'Вес в кг'),
        (r'\d+\s*л', 'Объем в литрах')
    ]
    
    for pattern, description in size_patterns:
        count = product_names.str.contains(pattern, case=False, regex=True).sum()
        if count > 0:
            print(f"  • {description}: {count:,} названий ({count/len(product_names)*100:.1f}%)")

=== АНАЛИЗ ТЕКСТОВЫХ ПАТТЕРНОВ ===
Названия с кириллицей: 130,296 (100.0%)
Названия с латиницей: 62,975 (48.3%)
Названия с цифрами: 123,353 (94.7%)
Названия со спецсимволами: 103,738 (79.6%)

=== ОБЩИЕ ПАТТЕРНЫ ===
  • Размеры (NxN): 16,895 названий (13.0%)
  • Миллиметры: 17,021 названий (13.1%)
  • Сантиметры: 1,489 названий (1.1%)
  • Метры: 11,078 названий (8.5%)
  • М-размер (М10, М12, и т.д.): 4,805 названий (3.7%)
  • Вес в кг: 1,732 названий (1.3%)
  • Объем в литрах: 3,968 названий (3.0%)


### 2.2 Распределение длины и сложности текста

In [9]:
if MAIN_NAME_COL:
    # Создание визуализаций для текстового анализа
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Распределение длины символов', 'Распределение количества слов',
                       'Распределение языковых паттернов', 'Топ-20 наиболее частых слов'),
        specs=[[{"type": "histogram"}, {"type": "histogram"}],
               [{"type": "pie"}, {"type": "bar"}]]
    )
    
    # Распределение длины символов
    fig.add_trace(
        go.Histogram(x=name_lengths, nbinsx=50, name='Длина символов',
                    marker_color='blue', opacity=0.7),
        row=1, col=1
    )
    
    # Распределение количества слов
    fig.add_trace(
        go.Histogram(x=word_counts, nbinsx=20, name='Количество слов',
                    marker_color='green', opacity=0.7),
        row=1, col=2
    )
    
    # Круговая диаграмма языковых паттернов
    mixed_count = sum(product_names.str.contains(cyrillic_pattern) & 
                     product_names.str.contains(latin_pattern))
    pattern_data = {
        'Только кириллица': cyrillic_count - mixed_count,
        'Только латиница': latin_count - mixed_count,
        'Смешанный': mixed_count,
        'Другое': len(product_names) - cyrillic_count - latin_count + mixed_count
    }
    
    fig.add_trace(
        go.Pie(labels=list(pattern_data.keys()), values=list(pattern_data.values()),
               name="Языковые паттерны"),
        row=2, col=1
    )
    
    # Наиболее частые слова
    all_words = ' '.join(product_names.str.lower()).split()
    # Удаление общих русских стоп-слов и коротких слов
    stop_words = {'и', 'в', 'на', 'с', 'по', 'для', 'из', 'от', 'до', 'при', 'без', 'под', 'над'}
    filtered_words = [word for word in all_words if len(word) > 2 and word not in stop_words]
    word_freq = Counter(filtered_words).most_common(20)
    
    words, counts = zip(*word_freq) if word_freq else ([], [])
    fig.add_trace(
        go.Bar(x=list(words), y=list(counts), name='Частота слов',
               marker_color='orange'),
        row=2, col=2
    )
    
    fig.update_layout(height=800, showlegend=False, 
                     title_text="Панель текстового анализа")
    fig.show()

### 2.3 Предобработка текста с модулями SAMe

In [ ]:
# Тестирование модулей обработки текста SAMe, если доступны
if SAME_MODULES_AVAILABLE and MAIN_NAME_COL:
    print("=== ТЕСТИРОВАНИЕ ОБРАБОТКИ ТЕКСТА SAMe ===")
    
    # Выборка названий товаров для тестирования
    sample_names = product_names.sample(min(100, len(product_names))).tolist()
    
    try:
        # Инициализация процессоров текста
        cleaner = TextCleaner(CleaningConfig())
        normalizer = TextNormalizer(NormalizerConfig())
        
        if SPACY_AVAILABLE:
            lemmatizer = Lemmatizer(LemmatizerConfig())
        
        # Обработка образцов названий
        processed_results = []
        
        for name in sample_names[:500]:  # Обработка первых 10 для демонстрации
            result = {
                'оригинал': name,
                'очищенный': cleaner.clean_text(name),
                'нормализованный': normalizer.normalize_text(name)
            }
            
            if SPACY_AVAILABLE:
                result['лемматизированный'] = lemmatizer.lemmatize_text(name)
            
            processed_results.append(result)
        
        # Отображение результатов
        processing_df = pd.DataFrame(processed_results)
        print("\nРезультаты обработки текста:")
        display(processing_df)
        
    except Exception as e:
        print(f"⚠️ Ошибка в обработке текста SAMe: {e}")
        
else:
    print("⚠️ Модули SAMe недоступны или основной столбец названий не найден")

=== ТЕСТИРОВАНИЕ ОБРАБОТКИ ТЕКСТА SAMe ===

Результаты обработки текста:


,оригинал,очищенный,нормализованный,лемматизированный
0,Фитинг гидравлический стальной резьба прямой #...,{'raw': 'Фитинг гидравлический стальной резьба...,{'original': 'Фитинг гидравлический стальной р...,{'original': 'Фитинг гидравлический стальной р...
1,Кольцо уплотнительное 305-320-85 ГОСТ 9833-73,{'raw': 'Кольцо уплотнительное 305-320-85 ГОСТ...,{'original': 'Кольцо уплотнительное 305-320-85...,{'original': 'Кольцо уплотнительное 305-320-85...
2,Тумба бу(24374-13025),"{'raw': 'Тумба бу(24374-13025)', 'html_cleaned...","{'original': 'Тумба бу(24374-13025)', 'units_n...","{'original': 'Тумба бу(24374-13025)', 'lemmati..."
3,Замок вис. дисковый ВС2-26 СР,"{'raw': 'Замок вис. дисковый ВС2-26 СР', 'html...","{'original': 'Замок вис. дисковый ВС2-26 СР', ...","{'original': 'Замок вис. дисковый ВС2-26 СР', ..."
4,Тяга 14006.55.151-01,"{'raw': 'Тяга 14006.55.151-01', 'html_cleaned'...","{'original': 'Тяга 14006.55.151-01', 'units_no...","{'original': 'Тяга 14006.55.151-01', 'lemmatiz..."
5,Уплотнение цилиндровой втулки 54025.53.385,{'raw': 'Уплотнение цилиндровой втулки 54025.5...,{'original': 'Уплотнение цилиндровой втулки 54...,{'original': 'Уплотнение цилиндровой втулки 54...
6,"Клапан челночный, HC01-07","{'raw': 'Клапан челночный, HC01-07', 'html_cle...","{'original': 'Клапан челночный, HC01-07', 'uni...","{'original': 'Клапан челночный, HC01-07', 'lem..."
7,Маркер белый для промышленной графики наконечн...,{'raw': 'Маркер белый для промышленной графики...,{'original': 'Маркер белый для промышленной гр...,{'original': 'Маркер белый для промышленной гр...
8,Шаблон полимерный под СБТ 114мм ТУ 3666-061-40...,{'raw': 'Шаблон полимерный под СБТ 114мм ТУ 36...,{'original': 'Шаблон полимерный под СБТ 114мм ...,{'original': 'Шаблон полимерный под СБТ 114мм ...
9,"Вагон-дом прачечная-душевая, на санях 20299","{'raw': 'Вагон-дом прачечная-душевая, на санях...","{'original': 'Вагон-дом прачечная-душевая, на ...","{'original': 'Вагон-дом прачечная-душевая, на ..."


Memory usage high: 80.4% of limit
High memory usage detected: 6.44GB
Memory usage high: 81.4% of limit
High memory usage detected: 6.51GB
Memory usage high: 81.1% of limit
High memory usage detected: 6.49GB
Memory usage high: 81.5% of limit
High memory usage detected: 6.52GB
Memory usage high: 87.0% of limit
High memory usage detected: 6.96GB
Memory usage high: 83.7% of limit
High memory usage detected: 6.70GB
Memory usage high: 83.8% of limit
High memory usage detected: 6.71GB
Memory usage high: 84.4% of limit
High memory usage detected: 6.75GB
Memory usage high: 84.9% of limit
High memory usage detected: 6.79GB
Memory usage high: 83.9% of limit
High memory usage detected: 6.71GB
Memory usage high: 85.0% of limit
High memory usage detected: 6.80GB
Memory usage high: 81.9% of limit
High memory usage detected: 6.55GB
Memory usage high: 82.1% of limit
High memory usage detected: 6.57GB
Memory usage high: 83.8% of limit
High memory usage detected: 6.70GB
Memory usage high: 88.0% of limit


## 3️⃣ Анализ метрик сходства и сопоставления

### 3.1 Сравнение алгоритмов строкового сходства

In [11]:
if MAIN_NAME_COL:
    # Выбор выборки для тестирования сходства
    sample_size = min(500, len(product_names))
    similarity_sample = product_names.sample(sample_size, random_state=42).tolist()
    
    print(f"=== СРАВНЕНИЕ АЛГОРИТМОВ СХОДСТВА ===")
    print(f"Тестирование на {len(similarity_sample)} названиях товаров")
    
    # Определение функций сходства
    similarity_functions = {
        'Левенштейн (RapidFuzz)': lambda x, y: fuzz.ratio(x, y) / 100,
        'Частичное соотношение': lambda x, y: fuzz.partial_ratio(x, y) / 100,
        'Соотношение сортировки токенов': lambda x, y: fuzz.token_sort_ratio(x, y) / 100,
        'Соотношение множества токенов': lambda x, y: fuzz.token_set_ratio(x, y) / 100,
        'FuzzyWuzzy соотношение': lambda x, y: fuzz_wuzzy.ratio(x, y) / 100
    }
    
    # Тестирование алгоритмов сходства на парах образцов
    import time
    
    # Генерация тестовых пар
    test_pairs = []
    for i in range(min(100, len(similarity_sample))):
        # Похожие пары (один товар с вариациями)
        base_name = similarity_sample[i]
        # Создание вариации путем добавления/удаления слов
        words = base_name.split()
        if len(words) > 1:
            variation = ' '.join(words[:-1])  # Удаление последнего слова
            test_pairs.append((base_name, variation, 'похожие'))
        
        # Непохожие пары
        if i < len(similarity_sample) - 1:
            test_pairs.append((similarity_sample[i], similarity_sample[i+1], 'разные'))
    
    # Бенчмарк функций сходства
    results = {}
    
    for func_name, func in similarity_functions.items():
        print(f"\nТестирование {func_name}...")
        
        start_time = time.time()
        similarities = []
        
        for pair1, pair2, pair_type in test_pairs[:50]:  # Тест на первых 50 парах
            try:
                sim_score = func(pair1, pair2)
                similarities.append((sim_score, pair_type))
            except Exception as e:
                print(f"  Ошибка с {func_name}: {e}")
                continue
        
        end_time = time.time()
        
        if similarities:
            # Расчет метрик производительности
            similar_scores = [s[0] for s in similarities if s[1] == 'похожие']
            different_scores = [s[0] for s in similarities if s[1] == 'разные']
            
            results[func_name] = {
                'среднее_время_сравнения': (end_time - start_time) / len(similarities),
                'средний_балл_похожих': np.mean(similar_scores) if similar_scores else 0,
                'средний_балл_разных': np.mean(different_scores) if different_scores else 0,
                'разделение_баллов': (np.mean(similar_scores) - np.mean(different_scores)) if similar_scores and different_scores else 0
            }
    
    # Отображение результатов
    if results:
        results_df = pd.DataFrame(results).T
        results_df = results_df.round(4)
        print("\n=== ПРОИЗВОДИТЕЛЬНОСТЬ АЛГОРИТМОВ СХОДСТВА ===")
        display(results_df)

=== СРАВНЕНИЕ АЛГОРИТМОВ СХОДСТВА ===
Тестирование на 500 названиях товаров

Тестирование Левенштейн (RapidFuzz)...

Тестирование Частичное соотношение...

Тестирование Соотношение сортировки токенов...

Тестирование Соотношение множества токенов...

Тестирование FuzzyWuzzy соотношение...

=== ПРОИЗВОДИТЕЛЬНОСТЬ АЛГОРИТМОВ СХОДСТВА ===


,среднее_время_сравнения,средний_балл_похожих,средний_балл_разных,разделение_баллов
Левенштейн (RapidFuzz),0.0,0.8564,0.2708,0.5856
Частичное соотношение,0.0,0.9978,0.3198,0.6780
Соотношение сортировки токенов,0.0,0.8572,0.2751,0.5820
Соотношение множества токенов,0.0,1.0000,0.2751,0.7249
FuzzyWuzzy соотношение,0.0,0.8568,0.2700,0.5868


### 3.2 Анализ семантического сходства

In [12]:
# Семантическое сходство с использованием TF-IDF и косинусного сходства
if MAIN_NAME_COL:
    print("=== АНАЛИЗ СЕМАНТИЧЕСКОГО СХОДСТВА ===")
    
    # Подготовка текста для TF-IDF
    semantic_sample = product_names.sample(min(1000, len(product_names)), random_state=42)
    
    # Создание TF-IDF векторов
    print("Создание TF-IDF векторов...")
    
    # TF-IDF на уровне символов
    char_vectorizer = TfidfVectorizer(
        analyzer='char',
        ngram_range=(2, 4),
        max_features=5000,
        lowercase=True
    )
    
    # TF-IDF на уровне слов
    word_vectorizer = TfidfVectorizer(
        analyzer='word',
        ngram_range=(1, 2),
        max_features=5000,
        lowercase=True,
        stop_words=list(stop_words)
    )
    
    try:
        char_tfidf = char_vectorizer.fit_transform(semantic_sample)
        word_tfidf = word_vectorizer.fit_transform(semantic_sample)
        
        print(f"Размер символьного TF-IDF: {char_tfidf.shape}")
        print(f"Размер словесного TF-IDF: {word_tfidf.shape}")
        
        # Расчет матриц сходства для небольшого подмножества
        subset_size = min(100, char_tfidf.shape[0])
        char_subset = char_tfidf[:subset_size]
        word_subset = word_tfidf[:subset_size]
        
        char_similarity = cosine_similarity(char_subset)
        word_similarity = cosine_similarity(word_subset)
        
        print(f"\n=== СТАТИСТИКА МАТРИЦЫ СХОДСТВА ===")
        print(f"Сходство на уровне символов:")
        print(f"  Среднее: {np.mean(char_similarity):.4f}")
        print(f"  Ст. откл.: {np.std(char_similarity):.4f}")
        print(f"  Максимум: {np.max(char_similarity):.4f}")
        
        print(f"\nСходство на уровне слов:")
        print(f"  Среднее: {np.mean(word_similarity):.4f}")
        print(f"  Ст. откл.: {np.std(word_similarity):.4f}")
        print(f"  Максимум: {np.max(word_similarity):.4f}")
        
        # Поиск наиболее похожих пар
        print(f"\n=== НАИБОЛЕЕ ПОХОЖИЕ ПАРЫ ТОВАРОВ ===")
        
        # Получение индексов наиболее похожих пар (исключая диагональ)
        np.fill_diagonal(char_similarity, 0)  # Удаление само-сходства
        top_indices = np.unravel_index(np.argsort(char_similarity.ravel())[-10:], char_similarity.shape)
        
        sample_list = semantic_sample.iloc[:subset_size].tolist()
        
        for i, (idx1, idx2) in enumerate(zip(top_indices[0][-5:], top_indices[1][-5:])):
            similarity_score = char_similarity[idx1, idx2]
            print(f"{i+1}. Сходство: {similarity_score:.4f}")
            print(f"   Товар 1: {sample_list[idx1]}")
            print(f"   Товар 2: {sample_list[idx2]}")
            print()
        
    except Exception as e:
        print(f"⚠️ Ошибка в анализе семантического сходства: {e}")

=== АНАЛИЗ СЕМАНТИЧЕСКОГО СХОДСТВА ===
Создание TF-IDF векторов...
Размер символьного TF-IDF: (1000, 5000)
Размер словесного TF-IDF: (1000, 5000)

=== СТАТИСТИКА МАТРИЦЫ СХОДСТВА ===
Сходство на уровне символов:
  Среднее: 0.0456
  Ст. откл.: 0.1052
  Максимум: 1.0000

Сходство на уровне слов:
  Среднее: 0.0132
  Ст. откл.: 0.1014
  Максимум: 1.0000

=== НАИБОЛЕЕ ПОХОЖИЕ ПАРЫ ТОВАРОВ ===
1. Сходство: 0.6224
   Товар 1: Подшипник 3518Н ГОСТ 5721-75
   Товар 2: Подшипник 50-92152М ГОСТ 520-89

2. Сходство: 0.6855
   Товар 1: Жилет утепленный цвет оранжевый р.56-58 рост 158-164
   Товар 2: Жилет утепленный цвет оранжевый р.68-70 рост 206-212

3. Сходство: 0.6855
   Товар 1: Жилет утепленный цвет оранжевый р.68-70 рост 206-212
   Товар 2: Жилет утепленный цвет оранжевый р.56-58 рост 158-164

4. Сходство: 0.8779
   Товар 1: Подшипник 3518Н ГОСТ 5721-75
   Товар 2: Подшипник 3609 ГОСТ 5721-75

5. Сходство: 0.8779
   Товар 1: Подшипник 3609 ГОСТ 5721-75
   Товар 2: Подшипник 3518Н ГОСТ 5721-7

## 4️⃣ Анализ категоризации и кластеризации

### 4.1 Анализ распределения категорий товаров

In [13]:
# Анализ категорий товаров, если доступны
category_analysis = {}

if CATEGORY_COL:
    print(f"=== АНАЛИЗ КАТЕГОРИЙ ({CATEGORY_COL}) ===")
    
    category_counts = df[CATEGORY_COL].value_counts()
    category_analysis['основные_категории'] = category_counts
    
    print(f"Всего категорий: {len(category_counts)}")
    print(f"Наиболее популярные категории:")
    for cat, count in category_counts.head(10).items():
        percentage = (count / len(df)) * 100
        print(f"  • {cat}: {count:,} товаров ({percentage:.1f}%)")
    
    # Покрытие категорий
    missing_categories = df[CATEGORY_COL].isnull().sum()
    coverage = ((len(df) - missing_categories) / len(df)) * 100
    print(f"\nПокрытие категорий: {coverage:.1f}% ({len(df) - missing_categories:,} товаров)")

if SUBCATEGORY_COL:
    print(f"\n=== АНАЛИЗ ПОДКАТЕГОРИЙ ({SUBCATEGORY_COL}) ===")
    
    subcategory_counts = df[SUBCATEGORY_COL].value_counts()
    category_analysis['подкатегории'] = subcategory_counts
    
    print(f"Всего подкатегорий: {len(subcategory_counts)}")
    print(f"Наиболее популярные подкатегории:")
    for subcat, count in subcategory_counts.head(10).items():
        percentage = (count / len(df)) * 100
        print(f"  • {subcat}: {count:,} товаров ({percentage:.1f}%)")
    
    # Покрытие подкатегорий
    missing_subcategories = df[SUBCATEGORY_COL].isnull().sum()
    subcoverage = ((len(df) - missing_subcategories) / len(df)) * 100
    print(f"\nПокрытие подкатегорий: {subcoverage:.1f}% ({len(df) - missing_subcategories:,} товаров)")

# Анализ иерархии категорий
if CATEGORY_COL and SUBCATEGORY_COL:
    print(f"\n=== АНАЛИЗ ИЕРАРХИИ КАТЕГОРИЙ ===")
    
    hierarchy = df.groupby([CATEGORY_COL, SUBCATEGORY_COL]).size().reset_index(name='количество')
    hierarchy = hierarchy.sort_values('количество', ascending=False)
    
    print(f"Всего комбинаций категория-подкатегория: {len(hierarchy)}")
    print(f"\nТоп-10 пар категория-подкатегория:")
    for _, row in hierarchy.head(10).iterrows():
        print(f"  • {row[CATEGORY_COL]} → {row[SUBCATEGORY_COL]}: {row['количество']:,} товаров")
    
    category_analysis['иерархия'] = hierarchy

=== АНАЛИЗ КАТЕГОРИЙ (Группа) ===
Всего категорий: 991
Наиболее популярные категории:
  • АВАНСОВЫЙ ОТЧЕТ: 14,704 товаров (11.3%)
  • Запасные части к грузовым автомобилям: 7,716 товаров (5.9%)
  • Запасные части к тракторам, специальной и дорожной технике: 5,175 товаров (4.0%)
  • Инструмент слесарно-монтажный: 3,717 товаров (2.9%)
  • Одежда специальная летняя: 2,886 товаров (2.2%)
  • Одежда специальная зимняя: 2,242 товаров (1.7%)
  • ЗИП к оборудованию Canrig: 1,621 товаров (1.2%)
  • ЗИП к оборудованию Tesco: 1,487 товаров (1.1%)
  • Переводники для бурильных колонн: 1,408 товаров (1.1%)
  • ЗИП к оборудованию National Oilwell Varco (NOV): 1,343 товаров (1.0%)

Покрытие категорий: 100.0% (130,303 товаров)

=== АНАЛИЗ ПОДКАТЕГОРИЙ (ВидНоменклатуры) ===
Всего подкатегорий: 39
Наиболее популярные подкатегории:
  • Запасные части: 49,468 товаров (38.0%)
  • Сырье и материалы: 23,577 товаров (18.1%)
  • Инвентарь и хозяйственные принадлежности: 20,321 товаров (15.6%)
  • Спецодежда: 1

### 4.2 Визуализация распределения категорий

In [14]:
# Визуализация распределения категорий
if category_analysis:
    fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=('Топ категории', 'Топ подкатегории', 
                       'Распределение размеров категорий', 'Тепловая карта иерархии'),
        specs=[[{"type": "bar"}], [{"type": "bar"}], 
               [{"type": "histogram"}], [{"type": "heatmap"}]]
    )
    
    # Топ категории
    if 'основные_категории' in category_analysis:
        top_cats = category_analysis['основные_категории'].head(15)
        fig.add_trace(
            go.Bar(x=top_cats.values, y=top_cats.index, orientation='h',
                   name='Категории', marker_color='blue'),
            row=1, col=1
        )
    
    # Топ подкатегории
    if 'подкатегории' in category_analysis:
        top_subcats = category_analysis['подкатегории'].head(15)
        fig.add_trace(
            go.Bar(x=top_subcats.values, y=top_subcats.index, orientation='h',
                   name='Подкатегории', marker_color='green'),
            row=2, col=1
        )
    
    # Распределение размеров категорий
    if 'основные_категории' in category_analysis:
        fig.add_trace(
            go.Histogram(x=category_analysis['основные_категории'].values,
                        nbinsx=30, name='Размеры категорий', marker_color='orange'),
            row=3, col=1
        )

    # Тепловая карта иерархии (если доступна)
    if 'иерархия' in category_analysis and len(category_analysis['иерархия']) > 0:
        # Создание сводной таблицы для тепловой карты
        hierarchy_pivot = category_analysis['иерархия'].pivot_table(
            index=CATEGORY_COL, columns=SUBCATEGORY_COL, values='количество', fill_value=0
        )
        
        # Ограничение топ категориями и подкатегориями для читаемости
        top_categories = category_analysis['основные_категории'].head(10).index
        top_subcategories = category_analysis['подкатегории'].head(15).index
        
        hierarchy_subset = hierarchy_pivot.loc[
            hierarchy_pivot.index.intersection(top_categories),
            hierarchy_pivot.columns.intersection(top_subcategories)
        ]
        
        fig.add_trace(
            go.Heatmap(z=hierarchy_subset.values,
                      x=hierarchy_subset.columns,
                      y=hierarchy_subset.index,
                      colorscale='Blues',
                      name='Иерархия'),
            row=4, col=1
        )
    
    fig.update_layout(height=800, showlegend=False,
                     title_text="Панель анализа категорий")
    fig.show()

## 5️⃣ Оптимизация производительности поиска

### 5.1 Рекомендации по структуре индексов

In [15]:
# Анализ оптимальных структур индексов для производительности поиска
print("=== АНАЛИЗ ОПТИМИЗАЦИИ ИНДЕКСОВ ПОИСКА ===")

if MAIN_NAME_COL:
    # Расчет статистики индексов
    total_products = len(df)
    unique_products = df[MAIN_NAME_COL].nunique()
    avg_name_length = df[MAIN_NAME_COL].str.len().mean()
    
    print(f"\n=== ОЦЕНКИ РАЗМЕРА ИНДЕКСОВ ===")
    print(f"Всего товаров: {total_products:,}")
    print(f"Уникальных товаров: {unique_products:,}")
    print(f"Средняя длина названия: {avg_name_length:.1f} символов")
    
    # Оценки памяти для разных типов индексов
    char_index_size = total_products * avg_name_length * 2  # UTF-8 кодировка
    tfidf_index_size = total_products * 5000 * 4  # Предполагая 5000 признаков, 4 байта на float
    
    print(f"\n=== ОЦЕНКИ ПАМЯТИ ===")
    print(f"Индекс сырого текста: {char_index_size / 1024**2:.1f} МБ")
    print(f"Индекс TF-IDF: {tfidf_index_size / 1024**2:.1f} МБ")
    
    # Рекомендации по производительности
    print(f"\n=== РЕКОМЕНДАЦИИ ПО ПРОИЗВОДИТЕЛЬНОСТИ ===")
    
    if total_products < 10000:
        print("📊 Малый датасет (<10К товаров):")
        print("  • Используйте TF-IDF в памяти с косинусным сходством")
        print("  • Простое нечеткое сопоставление для точного поиска")
        print("  • Нет необходимости в сложной индексации")
    elif total_products < 100000:
        print("📊 Средний датасет (10К-100К товаров):")
        print("  • Используйте FAISS для векторного поиска по сходству")
        print("  • Реализуйте n-граммную индексацию для нечеткого сопоставления")
        print("  • Рассмотрите предварительную фильтрацию по категориям")
    else:
        print("📊 Большой датасет (>100К товаров):")
        print("  • Используйте распределенный поиск (Elasticsearch/Solr)")
        print("  • Реализуйте иерархическую кластеризацию для предварительной фильтрации")
        print("  • Используйте приближенный поиск ближайших соседей")
    
    # Рекомендации по оптимизации запросов
    print(f"\n=== ОПТИМИЗАЦИЯ ЗАПРОСОВ ===")
    print("🔍 Рекомендуемый пайплайн поиска:")
    print("  1. Предобработка текста (очистка, нормализация)")
    print("  2. Фильтрация по категориям (если категория известна)")
    print("  3. Нечеткое сопоставление для точного поиска товаров")
    print("  4. Семантическое сходство для поиска аналогов")
    print("  5. Ранжирование результатов и дедупликация")
    
    # Рекомендации по кэшированию
    print(f"\n=== СТРАТЕГИЯ КЭШИРОВАНИЯ ===")
    print("💾 Рекомендуемое кэширование:")
    print("  • Кэширование TF-IDF векторов для частых товаров")
    print("  • Кэширование матриц сходства для кластеров товаров")
    print("  • Реализация кэширования результатов запросов с TTL")
    print("  • Использование Redis для распределенного кэширования")

=== АНАЛИЗ ОПТИМИЗАЦИИ ИНДЕКСОВ ПОИСКА ===

=== ОЦЕНКИ РАЗМЕРА ИНДЕКСОВ ===
Всего товаров: 130,303
Уникальных товаров: 128,937
Средняя длина названия: 49.0 символов

=== ОЦЕНКИ ПАМЯТИ ===
Индекс сырого текста: 12.2 МБ
Индекс TF-IDF: 2485.3 МБ

=== РЕКОМЕНДАЦИИ ПО ПРОИЗВОДИТЕЛЬНОСТИ ===
📊 Большой датасет (>100К товаров):
  • Используйте распределенный поиск (Elasticsearch/Solr)
  • Реализуйте иерархическую кластеризацию для предварительной фильтрации
  • Используйте приближенный поиск ближайших соседей

=== ОПТИМИЗАЦИЯ ЗАПРОСОВ ===
🔍 Рекомендуемый пайплайн поиска:
  1. Предобработка текста (очистка, нормализация)
  2. Фильтрация по категориям (если категория известна)
  3. Нечеткое сопоставление для точного поиска товаров
  4. Семантическое сходство для поиска аналогов
  5. Ранжирование результатов и дедупликация

=== СТРАТЕГИЯ КЭШИРОВАНИЯ ===
💾 Рекомендуемое кэширование:
  • Кэширование TF-IDF векторов для частых товаров
  • Кэширование матриц сходства для кластеров товаров
  • Реализа

## 6️⃣ Резюме и рекомендации

### 6.1 Резюме ключевых выводов

In [16]:
# Генерация комплексного резюме анализа
print("=== РЕЗЮМЕ АНАЛИЗА СИСТЕМЫ ПОИСКА АНАЛОГОВ SAMe ===")
print("=" * 60)

# Обзор датасета
if MAIN_NAME_COL and MAIN_NAME_COL in df.columns:
    # Очистка данных для точного подсчета
    clean_names_summary = df[MAIN_NAME_COL].dropna().astype(str)
    clean_names_summary = clean_names_summary[clean_names_summary.str.strip() != '']
    
    total_clean = len(clean_names_summary)
    unique_clean = clean_names_summary.nunique()
    duplicate_rate = ((total_clean - unique_clean) / total_clean * 100) if total_clean > 0 else 0
    avg_length = clean_names_summary.str.len().mean() if total_clean > 0 else 0
    
    print(f"\n📊 ОБЗОР ДАТАСЕТА:")
    print(f"  • Всего товаров: {len(df):,}")
    print(f"  • Товаров с названиями: {total_clean:,}")
    print(f"  • Уникальных названий товаров: {unique_clean:,}")
    print(f"  • Средняя длина названия: {avg_length:.1f} символов")
    print(f"  • Процент дубликатов: {duplicate_rate:.1f}%")

# Анализ категорий
if CATEGORY_COL:
    print(f"\n🏷️ КАТЕГОРИЗАЦИЯ:")
    print(f"  • Основных категорий: {df[CATEGORY_COL].nunique()}")
    print(f"  • Покрытие категорий: {((len(df) - df[CATEGORY_COL].isnull().sum()) / len(df) * 100):.1f}%")
    if SUBCATEGORY_COL:
        print(f"  • Подкатегорий: {df[SUBCATEGORY_COL].nunique()}")
        print(f"  • Покрытие подкатегорий: {((len(df) - df[SUBCATEGORY_COL].isnull().sum()) / len(df) * 100):.1f}%")

# Выводы текстового анализа
if MAIN_NAME_COL and MAIN_NAME_COL in df.columns:
    # Используем уже очищенные данные
    if 'clean_names_summary' in locals() and len(clean_names_summary) > 0:
        text_data = clean_names_summary
    else:
        text_data = df[MAIN_NAME_COL].dropna().astype(str)
        text_data = text_data[text_data.str.strip() != '']
    
    if len(text_data) > 0:
        cyrillic_pct = (text_data.str.contains(r'[а-яё]', case=False, regex=True).sum() / len(text_data)) * 100
        numeric_pct = (text_data.str.contains(r'\d', regex=True).sum() / len(text_data)) * 100
        avg_words = text_data.str.split().str.len().mean()
        
        print(f"\n📝 ХАРАКТЕРИСТИКИ ТЕКСТА:")
        print(f"  • Кириллический текст: {cyrillic_pct:.1f}% товаров")
        print(f"  • Содержит цифры: {numeric_pct:.1f}% товаров")
        print(f"  • Среднее количество слов в названии: {avg_words:.1f}")
    else:
        print(f"\n📝 ХАРАКТЕРИСТИКИ ТЕКСТА: Нет данных для анализа")

# Рекомендации по производительности
print(f"\n⚡ РЕКОМЕНДАЦИИ ПО ПРОИЗВОДИТЕЛЬНОСТИ:")
if len(df) < 10000:
    print(f"  • Размер датасета: Малый (<10К) - Используйте обработку в памяти")
    print(f"  • Рекомендуется: TF-IDF + Косинусное сходство")
elif len(df) < 100000:
    print(f"  • Размер датасета: Средний (10К-100К) - Используйте FAISS индексацию")
    print(f"  • Рекомендуется: Гибридный нечеткий + семантический поиск")
else:
    print(f"  • Размер датасета: Большой (>100К) - Используйте распределенный поиск")
    print(f"  • Рекомендуется: Elasticsearch с пользовательскими анализаторами")

print(f"\n🎯 ПОРОГИ СХОДСТВА:")
print(f"  • Высокая точность (точные совпадения): ≥ 0.90")
print(f"  • Сбалансированный (похожие товары): ≥ 0.80")
print(f"  • Высокая полнота (широкое сопоставление): ≥ 0.70")

print(f"\n🔧 ПРИОРИТЕТЫ РЕАЛИЗАЦИИ:")
print(f"  1. Реализовать надежный пайплайн предобработки текста")
print(f"  2. Создать фильтры поиска с учетом категорий")
print(f"  3. Построить гибридный движок нечеткого + семантического сходства")
print(f"  4. Реализовать ранжирование результатов и дедупликацию")
print(f"  5. Добавить мониторинг производительности и оптимизацию")

print(f"\n✅ АНАЛИЗ ЗАВЕРШЕН УСПЕШНО!")
print(f"📋 Данный анализ обеспечивает основу для построения эффективной системы поиска аналогов.")

=== РЕЗЮМЕ АНАЛИЗА СИСТЕМЫ ПОИСКА АНАЛОГОВ SAMe ===

📊 ОБЗОР ДАТАСЕТА:
  • Всего товаров: 130,303
  • Товаров с названиями: 130,303
  • Уникальных названий товаров: 128,937
  • Средняя длина названия: 49.0 символов
  • Процент дубликатов: 1.0%

🏷️ КАТЕГОРИЗАЦИЯ:
  • Основных категорий: 991
  • Покрытие категорий: 100.0%
  • Подкатегорий: 39
  • Покрытие подкатегорий: 100.0%

📝 ХАРАКТЕРИСТИКИ ТЕКСТА:
  • Кириллический текст: 100.0% товаров
  • Содержит цифры: 94.7% товаров
  • Среднее количество слов в названии: 6.3

⚡ РЕКОМЕНДАЦИИ ПО ПРОИЗВОДИТЕЛЬНОСТИ:
  • Размер датасета: Большой (>100К) - Используйте распределенный поиск
  • Рекомендуется: Elasticsearch с пользовательскими анализаторами

🎯 ПОРОГИ СХОДСТВА:
  • Высокая точность (точные совпадения): ≥ 0.90
  • Сбалансированный (похожие товары): ≥ 0.80
  • Высокая полнота (широкое сопоставление): ≥ 0.70

🔧 ПРИОРИТЕТЫ РЕАЛИЗАЦИИ:
  1. Реализовать надежный пайплайн предобработки текста
  2. Создать фильтры поиска с учетом категорий
  3. 